In [11]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from requests import get
import time
import pickle
import os
import json
from typing import Dict, List
import requests
import pprint



import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

import prep
from env import github_token

## Planning

## Acquire 

Acuired most starred repos using BeautifulSoup to scrape repo links from the first 22 pages. Then used the Github API to pull the README.md files for each repo.

In [2]:
#grabs list of repo names from most starred repo pages.
def get_urls(start_page,stop_page):
    pages = list(range(start_page,stop_page+1))
    page_urls = []
    for page in pages:
        p = f'https://github.com/search?o=desc&p={page}&q=stars%3A%3E1&s=stars&type=Repositories'
        page_urls.append(p)

    repo_links_all = []
    time.sleep(3)
    for page in page_urls:
        url = page
        headers  = {'User-Agent' : 'Codeup Data Science Student'} #user-agent to decrease chance of being rejected by github
        response = get(url,headers)
        soup = BeautifulSoup(response.text) #created a BS object
        repo_list = soup.find('ul', class_='repo-list') #find section of text of interest
        links = repo_list.find_all(href = True) #isolate the 'href's 
        repo_links= [link['href'] for link in links if not re.match(r'^/topics',link['href']) and not re.match(r'^https',link['href'])\
                 and link['href'].count('/') < 3] #further pare down list by excluding anything that begins with /topics, https or more than 2 forward slashes
        repo_links_all.extend(repo_links)
    repo_links_all = [url[1:] for url in repo_links_all]
    time.sleep(2)
    return repo_links_all

We had problems scraping the website with the rest of the class also pinging the Github servers so we decided to save our urls and READMEs as files to be accessed locally

In [4]:
with open ('urls', 'rb') as fp:
    urls = pickle.load(fp)

In [7]:
df = pd.read_json('data.json')
df.head(2)

,language,readme_contents,repo
0,JavaScript,![freeCodeCamp.org Social Banner](https://s3.a...,freeCodeCamp/freeCodeCamp
1,Rust,[996.ICU](https://996.icu/#/en_US)\n=======\n*...,996icu/996.ICU


### Initial Exploration

We pulled 220 initial README files and grouped by the language. 32 had no language type and were dropped. After further discussion we decided to only work on classifying the top 5 most common languages of our data.

In [13]:
pprint.pprint(df.groupby('language')['repo'].count().sort_values(ascending=False).head(8))

language
JavaScript    72
Python        25
Java          17
C++           12
Go            12
TypeScript     9
CSS            5
C              4
Name: repo, dtype: int64
